In [1]:
tickers = ['WEGE3.SA', 'BBAS3.SA']
periodo = '2y'  # Opções: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

In [2]:
!pip install -q yfinance

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.graph_objects as go

from ipywidgets import interact

In [4]:
def fix_col_names(df):
  return ['IBOV' if col =='^BVSP' else col.rstrip('.SA') for col in df.columns]

In [5]:
prices = yf.download(tickers, period=periodo, rounding=True )['Adj Close']
prices.columns = fix_col_names(prices)
prices.dropna(inplace=True)
retorno = prices.pct_change().dropna()
ativos = retorno.columns.to_list()

[*********************100%***********************]  2 of 2 completed


In [6]:
def calcula_carteira(df, w1):
  pesos = [w1,(1-w1)]  
  df2 = df.dot(pesos).copy()
  return  df2.mean() * 252, df2.std() * np.sqrt(252)

In [7]:
# Criando 100 carteiras de dois ativos com pesos variáveis
carteira = pd.DataFrame()
for i in np.linspace(0,1, 101):
  media, std = calcula_carteira(retorno,i)
  carteira.at[i,'retorno'] = media
  carteira.at[i,'volatilidade'] = std

In [8]:
# Carteira de volatilidade mínima
min_vol_idx = carteira['volatilidade'].idxmin()

In [9]:
# Calcula a fronteira eficiente
if (carteira['retorno'].iloc[0] > carteira['retorno'].iloc[-1]):
  fe = carteira.loc[:min_vol_idx, :]
else:
  fe = carteira.loc[min_vol_idx:, :]

In [10]:
def gerar_grafico(w1, mostrar_curvas):
  fig = go.Figure()

  # Desenha um ponto com o retorno e a volatilidade da carteira
  fig.add_scatter(y=[carteira.iloc[w1]['retorno']], 
                  x=[carteira.iloc[w1]['volatilidade']], 
                  text=['CARTEIRA'],
                  mode='markers+text', name='CARTEIRA')
  
  # Desenha os pontos das ações individuais
  fig.add_scatter(y=carteira.iloc[[-1,0]]['retorno'], 
                  x=carteira.iloc[[-1,0]]['volatilidade'], 
                  text=ativos,
                  mode='markers+text', name='Ações')
  
  # Desenha a curva
  fig.add_scatter(y=carteira['retorno'], 
                  x=carteira['volatilidade'],                  
                  mode='lines', name='Curva', 
                  visible=mostrar_curvas)

  # Plota carteira de volatilidade mínima   
  fig.add_scatter(y=[carteira.loc[min_vol_idx]['retorno']], 
                  x=[carteira.loc[min_vol_idx]['volatilidade']],                  
                  mode='markers', 
                  name='Carteira de Mínima Variância',
                  visible=mostrar_curvas)
      
  # Desenha a fronteira eficiente
  fig.add_scatter(y=fe['retorno'], 
                  x=fe['volatilidade'],                  
                  mode='lines', name='Fronteira Eficiente', 
                  line={'color':'red'},
                  visible=mostrar_curvas)
                
  fig.update_traces(textfont_size=12, 
                  textposition='top center', 
                  textfont_color='white',
                  hovertemplate='<b>retorno: </b> %{y:.1%}'+
                                '<br><b>volatilidade:</b> %{x:.1%}')
  fig.layout.autosize = False
  fig.layout.xaxis.title='Volatilidade'
  fig.layout.yaxis.title='Retorno Esperado'
  fig.layout.xaxis.range = [carteira['volatilidade'].min()-0.05, carteira['volatilidade'].max()+0.05]
  fig.layout.yaxis.range = [carteira['retorno'].min()-0.05, carteira['retorno'].max()+0.05]
  fig.layout.xaxis.tickformat = '.0%'
  fig.layout.yaxis.tickformat = '.0%'
  fig.layout.title = f"<b>{ativos[0]}:</b> {w1}%  <b>{ativos[1]}:</b> {100-w1}%"
  fig.layout.template = 'plotly_dark'
                   
  fig.show(config=dict(
                    displayModeBar=True
                ))

interact(gerar_grafico, w1=(0,100, 1), mostrar_curvas=False);

interactive(children=(IntSlider(value=50, description='w1'), Checkbox(value=False, description='mostrar_curvas…